# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbb586eed60>
├── 'a' --> tensor([[ 0.0049,  0.7568,  0.7674],
│                   [ 0.2551,  0.3819, -0.1944]])
└── 'x' --> <FastTreeValue 0x7fbb586b20a0>
    └── 'c' --> tensor([[ 0.9028,  0.6061,  0.1398,  1.2202],
                        [-1.1538, -0.2589, -1.1863, -0.0037],
                        [-1.9665, -0.0657, -0.9069,  1.3180]])

In [4]:
t.a

tensor([[ 0.0049,  0.7568,  0.7674],
        [ 0.2551,  0.3819, -0.1944]])

In [5]:
%timeit t.a

64.5 ns ± 0.0402 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbb586eed60>
├── 'a' --> tensor([[ 0.9355,  2.5355, -0.1965],
│                   [-0.4760, -0.1809,  0.9122]])
└── 'x' --> <FastTreeValue 0x7fbb586b20a0>
    └── 'c' --> tensor([[ 0.9028,  0.6061,  0.1398,  1.2202],
                        [-1.1538, -0.2589, -1.1863, -0.0037],
                        [-1.9665, -0.0657, -0.9069,  1.3180]])

In [7]:
%timeit t.a = new_value

67.1 ns ± 0.0527 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0049,  0.7568,  0.7674],
               [ 0.2551,  0.3819, -0.1944]]),
    x: Batch(
           c: tensor([[ 0.9028,  0.6061,  0.1398,  1.2202],
                      [-1.1538, -0.2589, -1.1863, -0.0037],
                      [-1.9665, -0.0657, -0.9069,  1.3180]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0049,  0.7568,  0.7674],
        [ 0.2551,  0.3819, -0.1944]])

In [11]:
%timeit b.a

57.4 ns ± 0.0759 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1042,  1.9066, -1.4315],
               [-1.2867, -0.6361, -1.6067]]),
    x: Batch(
           c: tensor([[ 0.9028,  0.6061,  0.1398,  1.2202],
                      [-1.1538, -0.2589, -1.1863, -0.0037],
                      [-1.9665, -0.0657, -0.9069,  1.3180]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.127 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

835 ns ± 2.44 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 560 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 1.81 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbaaffbb6d0>
├── 'a' --> tensor([[[ 0.0049,  0.7568,  0.7674],
│                    [ 0.2551,  0.3819, -0.1944]],
│           
│                   [[ 0.0049,  0.7568,  0.7674],
│                    [ 0.2551,  0.3819, -0.1944]],
│           
│                   [[ 0.0049,  0.7568,  0.7674],
│                    [ 0.2551,  0.3819, -0.1944]],
│           
│                   [[ 0.0049,  0.7568,  0.7674],
│                    [ 0.2551,  0.3819, -0.1944]],
│           
│                   [[ 0.0049,  0.7568,  0.7674],
│                    [ 0.2551,  0.3819, -0.1944]],
│           
│                   [[ 0.0049,  0.7568,  0.7674],
│                    [ 0.2551,  0.3819, -0.1944]],
│           
│                   [[ 0.0049,  0.7568,  0.7674],
│                    [ 0.2551,  0.3819, -0.1944]],
│           
│                   [[ 0.0049,  0.7568,  0.7674],
│                    [ 0.2551,  0.3819, -0.1944]]])
└── 'x' --> <FastTreeValue 0x7fbaafec1b80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 48.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbaaffbb910>
├── 'a' --> tensor([[ 0.0049,  0.7568,  0.7674],
│                   [ 0.2551,  0.3819, -0.1944],
│                   [ 0.0049,  0.7568,  0.7674],
│                   [ 0.2551,  0.3819, -0.1944],
│                   [ 0.0049,  0.7568,  0.7674],
│                   [ 0.2551,  0.3819, -0.1944],
│                   [ 0.0049,  0.7568,  0.7674],
│                   [ 0.2551,  0.3819, -0.1944],
│                   [ 0.0049,  0.7568,  0.7674],
│                   [ 0.2551,  0.3819, -0.1944],
│                   [ 0.0049,  0.7568,  0.7674],
│                   [ 0.2551,  0.3819, -0.1944],
│                   [ 0.0049,  0.7568,  0.7674],
│                   [ 0.2551,  0.3819, -0.1944],
│                   [ 0.0049,  0.7568,  0.7674],
│                   [ 0.2551,  0.3819, -0.1944]])
└── 'x' --> <FastTreeValue 0x7fbb2cb7f580>
    └── 'c' --> tensor([[ 0.9028,  0.6061,  0.1398,  1.2202],
                        [-1.1538, -0.2589, -1.1863, -0.0037],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 48.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.6 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.9028,  0.6061,  0.1398,  1.2202],
                       [-1.1538, -0.2589, -1.1863, -0.0037],
                       [-1.9665, -0.0657, -0.9069,  1.3180]],
              
                      [[ 0.9028,  0.6061,  0.1398,  1.2202],
                       [-1.1538, -0.2589, -1.1863, -0.0037],
                       [-1.9665, -0.0657, -0.9069,  1.3180]],
              
                      [[ 0.9028,  0.6061,  0.1398,  1.2202],
                       [-1.1538, -0.2589, -1.1863, -0.0037],
                       [-1.9665, -0.0657, -0.9069,  1.3180]],
              
                      [[ 0.9028,  0.6061,  0.1398,  1.2202],
                       [-1.1538, -0.2589, -1.1863, -0.0037],
                       [-1.9665, -0.0657, -0.9069,  1.3180]],
              
                      [[ 0.9028,  0.6061,  0.1398,  1.2202],
                       [-1.1538, -0.2589, -1.1863, -0.0037],
                       [-1.9665, -0.0657, -0.9069,  1.3180]],

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.9028,  0.6061,  0.1398,  1.2202],
                      [-1.1538, -0.2589, -1.1863, -0.0037],
                      [-1.9665, -0.0657, -0.9069,  1.3180],
                      [ 0.9028,  0.6061,  0.1398,  1.2202],
                      [-1.1538, -0.2589, -1.1863, -0.0037],
                      [-1.9665, -0.0657, -0.9069,  1.3180],
                      [ 0.9028,  0.6061,  0.1398,  1.2202],
                      [-1.1538, -0.2589, -1.1863, -0.0037],
                      [-1.9665, -0.0657, -0.9069,  1.3180],
                      [ 0.9028,  0.6061,  0.1398,  1.2202],
                      [-1.1538, -0.2589, -1.1863, -0.0037],
                      [-1.9665, -0.0657, -0.9069,  1.3180],
                      [ 0.9028,  0.6061,  0.1398,  1.2202],
                      [-1.1538, -0.2589, -1.1863, -0.0037],
                      [-1.9665, -0.0657, -0.9069,  1.3180],
                      [ 0.9028,  0.6061,  0.1398,  1.2202],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

323 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
